In [122]:
#MessageEvent.csv
def extract_message(df, windows, timestamp):
    #messages sent / received / total texts
    windowed_messageevent = {}
    before_esm = df[df.index <= timestamp]
    for window_name, window_size in windows.items():
        window_timedelta = pd.Timedelta(seconds=window_size)
        windowed_data = before_esm[before_esm.index >= (timestamp - window_timedelta)]
        unique_numbers_outgoing = []
        unique_numbers_incoming = []
        messages_outgoing = 0
        messages_incoming = 0 
        unique_messegers_outgoing = 0
        unique_messegers_incoming = 0 
        total_messages = 0 
        for message_time, message_type, number, in windowed_data[['messageBox', 'number']].itertuples(index=True):
            total_messages += 1
            if message_type == 'SENT':
                messages_outgoing += 1
                if number not in unique_numbers_outgoing:
                    unique_messegers_outgoing += 1
                    unique_numbers_outgoing.append(number)
            elif message_type == 'INBOX':
                messages_incoming += 1
                if number not in unique_numbers_incoming:
                    unique_messegers_incoming += 1
                    unique_numbers_incoming.append(number)
        windowed_messageevent[f'{window_name}_unique_messegers_outgoing'] = unique_messegers_outgoing
        windowed_messageevent[f'{window_name}_unique_messegers_incoming'] = unique_messegers_incoming
        windowed_messageevent[f'{window_name}_total_messages'] = total_messages
        
    return windowed_messageevent

In [123]:
#DeviceEvent.csv
def extract_deviceevent(df, windows, timestamp):
    # Number of times a phone is unlocked / time spent on phone
    windowed_deviceevent = {}
    before_esm = df[df.index <= timestamp]
    for window_name, window_size in windows.items():
        window_timedelta = pd.Timedelta(seconds=window_size)
        windowed_data = before_esm[before_esm.index >= (timestamp - window_timedelta)]
        times_unlocked = 0
        time_spent_on_phone = 0
        unlock_time = None  # Variable to store the timestamp of the last unlock event
        for event_time, event_type in windowed_data[['type']].itertuples(index=True):
            if event_type == 'UNLOCK':
                times_unlocked += 1
                unlock_time = event_time  # Update the unlock time
            
            elif event_type == 'SCREEN_OFF' and unlock_time is not None:
                # Calculate the time spent on phone by subtracting unlock time from screen off time
                time_spent_on_phone += (event_time - unlock_time).total_seconds()
                unlock_time = None  # Reset the unlock time
        proportion_time_spent_on_phone = time_spent_on_phone / window_size
        # Store the calculated values in the windowed_deviceevent dictionary
        windowed_deviceevent[f'{window_name}_times_unlocked'] = times_unlocked
        windowed_deviceevent[f'{window_name}_proportion_of_time_spent_on_phone'] = proportion_time_spent_on_phone
    
    return windowed_deviceevent

In [2]:
#AppUsageEvent
def entropy(labels):
    n_labels = len(labels)
    
    if n_labels <= 1:
        return 0
    
    _, counts = np.unique(labels, return_counts=True)
    probs = counts / n_labels
    entropy = -np.sum(probs * np.log2(probs))
    
    return entropy

def extract_appusage(df, windows, timestamp):
    windowed_appevent = {}
    before_esm = df[df.index <= timestamp]
    
    for window_name, window_size in windows.items():
        window_timedelta = pd.Timedelta(seconds=window_size)
        window_start = timestamp - window_timedelta
        windowed_data = before_esm[(before_esm.index >= window_start) & (before_esm.index <= timestamp)]
        
        # Count the number of events associated with each app category
        app_category_counts = windowed_data['category'].value_counts().head(5)
        
        # Calculate the total number of app interactions in the window
        total_interactions = len(windowed_data)
        
        # Calculate the proportion of time spent using each app category
        app_category_proportions = app_category_counts / total_interactions
        
        # Calculate the entropy of the app category distribution
        app_category_entropy = entropy(windowed_data['category'])
        
        # Create keys for each category in the windowed_appevent dictionary
        for category, count in app_category_counts.items():
            windowed_appevent[f'{window_name}_{category}_number_of_events'] = count
            #windowed_appevent[f'{window_name}_{category}_proportion_of_time'] = app_category_proportions[category]
        
        # Add entropy to the windowed_appevent dictionary
        windowed_appevent[f'{window_name}_category_entropy'] = app_category_entropy
        
    return windowed_appevent

In [125]:
#CallEvent.csv
def extract_call(df, windows, timestamp):
    #messages sent / received / total texts
    windowed_callevent = {}
    before_esm = df[df.index <= timestamp]
    for window_name, window_size in windows.items():
        window_timedelta = pd.Timedelta(seconds=window_size)
        windowed_data = before_esm[before_esm.index >= (timestamp - window_timedelta)]
        unique_numbers_outgoing = []
        unique_numbers_incoming = []
        unique_callers_outgoing = 0
        unique_callers_incoming = 0 
        time_spent_calling = 0
        for call_time, number, call_type, duration in windowed_data[['number', 'type', 'duration']].itertuples(index=True): 
            time_spent_calling += duration
            if call_type == 'OUTGOING':
                if number not in unique_numbers_outgoing:
                    unique_callers_outgoing += 1
                    unique_numbers_outgoing.append(number)
            elif call_type == 'INCOMING':
                if number not in unique_numbers_incoming:
                    unique_callers_incoming += 1
                    unique_numbers_incoming.append(number)
        windowed_callevent[f'{window_name}_unique_callers_outgoing'] = unique_callers_outgoing
        windowed_callevent[f'{window_name}_unique_callers_incoming'] = unique_callers_incoming
        windowed_callevent[f'{window_name}_time_spent_calling'] = time_spent_calling
    return windowed_callevent

In [5]:
# #Location.csv
# def calculate_entropy(cluster_counts):
#     total_time = cluster_counts.sum()
#     cluster_proportions = cluster_counts / total_time
#     entropy = -np.sum([p * np.log2(p) for p in cluster_proportions.values if p != 0])
#     return entropy


# def extract_location(df, windows, timestamp):
#     windowed_location = {}
#     before_esm = df[df.index <= timestamp]
#     # Iterate through each window
#     for window_name, window_size in windows.items():
#         window_timedelta = pd.Timedelta(seconds=window_size)
#         windowed_data = before_esm[before_esm.index >= (timestamp - window_timedelta)]
#         # Calculate cluster counts within the window
#         cluster_counts = windowed_data['cluster'].value_counts()
        
#         # Calculate entropy for the window
#         window_entropy = calculate_entropy(cluster_counts)
        
#         # Store entropy in the windowed_location dictionary
#         windowed_location[f'{window_name}_location_entropy'] = window_entropy
    
#     return windowed_location

def calculate_entropy(cluster_counts):
    total_time = cluster_counts.sum()
    
    if total_time == 0:
        return 0  # Return zero entropy if there are no counts

    cluster_proportions = cluster_counts / total_time
    
    # Filter out zero proportions
    cluster_proportions = cluster_proportions[cluster_proportions > 0]
    
    # Calculate entropy
    entropy = -np.sum(cluster_proportions * np.log2(cluster_proportions))
    
    return entropy

# Define the extract_location function
def extract_location(df, windows, timestamp):
    windowed_location = {}
    before_esm = df[df.index <= timestamp]
    
    # Iterate through each window
    for window_name, window_size in windows.items():
        window_timedelta = pd.Timedelta(seconds=window_size)
        windowed_data = before_esm[before_esm.index >= (timestamp - window_timedelta)]
        
        # Calculate cluster counts within the window
        cluster_counts = windowed_data['cluster'].value_counts()
        
        # Calculate entropy for the window
        window_entropy = calculate_entropy(cluster_counts)
        
        # Store entropy in the windowed_location dictionary
        windowed_location[f'{window_name}_location_entropy'] = window_entropy
    
    return windowed_location

In [1]:
def generic_entropy(data):
    value_counts = data.value_counts()
    probabilities = value_counts / len(data)
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy

def extract_generic_window_features(df, windows, timestamp):
    windowed_features = {}
    before_esm = df[df.index <= timestamp]
        
    for window_name, window_size in windows.items():
        window_timedelta = pd.Timedelta(seconds=window_size)
        windowed_data = before_esm[before_esm.index >= (timestamp - window_timedelta)]

        # Calculate mean, median, and standard deviation for numeric columns
        numeric_cols = windowed_data.select_dtypes(include=np.number).columns
        for col in numeric_cols:
            col_mean = windowed_data[col].mean()
            col_median = windowed_data[col].median()
            col_std = windowed_data[col].std()
            col_entropy = calculate_entropy(windowed_data[col])
            windowed_features[f'{window_name}_{col}_mean'] = col_mean
            windowed_features[f'{window_name}_{col}_median'] = col_median
            windowed_features[f'{window_name}_{col}_std'] = col_std
            windowed_features[f'{window_name}_{col}_entropy'] = col_entropy
    
    return windowed_features

In [3]:
def extract_windowed_data(all_participants_data, esm_responses, user_info, processing_functions, generic_windows):
    windowed_data_list = []  # List to store dictionaries of windowed features
    
    esm_responses = pd.merge(user_info, esm_responses, on='Pcode')
    esm_responses.sort_values(by=['Pcode', 'ResponseTime'], inplace=True)
    esm_responses.set_index('ResponseTime', inplace = True)
    
    # Iterate through each participant's data dictionary
    for participant_id, participant_data in all_participants_data.items():
        # Get all ESM responses for the current participant
        participant_esm_responses = esm_responses[esm_responses['Pcode'] == participant_id]
        
        # Iterate through each ESM response for the participant
        for index, esm_response in participant_esm_responses.iterrows():
            timestamp = index
            timestamp = pd.Timestamp(timestamp)
            p_code = esm_response['Pcode']
            stress = esm_response['Stress_binary']
            valence = esm_response['Valence_binary']
            arousal = esm_response['Arousal_binary']
            user_info_row = user_info[user_info['Pcode'] == participant_id].iloc[0]
            age = user_info_row['Age']
            gender = user_info_row['Gender']
            openness = user_info_row['Openness']
            conscientiousness = user_info_row['Conscientiousness']
            neuroticism = user_info_row['Neuroticism']
            extraversion = user_info_row['Extraversion']
            agreeableness = user_info_row['Agreeableness']
            pss10 = user_info_row['PSS10']
            phq9 = user_info_row['PHQ9']
            ghq12 = user_info_row['GHQ12']     
        
            
            windowed_features = {
                'ResponseTime': timestamp,
                'Pcode': p_code,
                'Stress_binary': stress,
                'Valence_binary': valence,
                'Arousal_binary': arousal,
                'Age': age,
                'Gender': gender,
                'Openness': openness,
                'Conscientiousness': conscientiousness,
                'Neuroticism': neuroticism,
                'Extraversion': extraversion,
                'Agreeableness': agreeableness,
                'PSS_last_week': pss10,
                'PHQ_last_week': phq9,
                'GHQ_last_week': ghq12
                
            }
            
            for dataframe_name, dataframe in participant_data.items():
                if dataframe_name == 'sleep_proxies':
                    continue
                if dataframe_name in processing_functions.keys():
                    proc_function, proc_windows = processing_functions[dataframe_name]
                    additional_features = proc_function(dataframe, proc_windows, timestamp)
                    windowed_features.update(additional_features)

                else:
                    windows = generic_windows
                    generic_features = extract_generic_window_features(dataframe, windows, timestamp)
                    windowed_features.update(generic_features)
                
            windowed_data_list.append(windowed_features)
    
 
    windowed_data_df = pd.DataFrame(windowed_data_list)
    windowed_data_df.set_index(['ResponseTime', 'Pcode'], inplace = True)
    return windowed_data_df

In [2]:
def add_sleep_proxy(all_participants_data, windowed_df):
    windowed_df['previous_nights_sleep_proxy'] = np.NaN
    for (timestamp, Pcode), row in windowed_df.iterrows():
        sleep_proxies = all_participants_data[Pcode]['sleep_proxies']
        for date, sleep_proxy in sleep_proxies.iterrows():
            if timestamp.date() == date:
                windowed_df.loc[(timestamp, Pcode), 'previous_nights_sleep_proxy'] = sleep_proxy['SleepProxy']